<a href="https://colab.research.google.com/github/bypolyarka/home-credit-kaggle/blob/master/home_credit_full_data_fe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Данные из других таблиц

* application_train/application_test: Основные данные, заемщик идентифицируется по полю SK_ID_CURR

* bureau: Данные по предыдущим займам в других кредитных организациях из кредитного бюро

* bureau_balance: Ежемесячные данные по предыдущим кредитам по бюро. Каждая строка — месяц испльзования кредита

* previous_application: Предыдущие заявки по кредитам в Home Credit, каждая имеет уникальное поле SK_ID_PREV

* POS_CASH_BALANCE: Ежемесячные данные по кредитам в Home Creditс выдачей наличными и кредитам на покупки товаров

* credit_card_balance: Ежемесячные данные по балансу кредитных карт в Home Credit

* installments_payment: Платежная история предыдущих займов в Home Credit.

In [1]:
import gc
import pandas as pd
import numpy as np


from copy import copy

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder

from lightgbm import LGBMClassifier

In [2]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/input/application_train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/input/application_test.csv')
prev = pd.read_csv('/content/drive/My Drive/Colab Notebooks/input/previous_application.csv')
buro = pd.read_csv('/content/drive/My Drive/Colab Notebooks/input/bureau.csv')
buro_balance = pd.read_csv('/content/drive/My Drive/Colab Notebooks/input/bureau_balance.csv')
credit_card  = pd.read_csv('/content/drive/My Drive/Colab Notebooks/input/credit_card_balance.csv')
POS_CASH  = pd.read_csv('/content/drive/My Drive/Colab Notebooks/input/POS_CASH_balance.csv')
payments = pd.read_csv('/content/drive/My Drive/Colab Notebooks/input/installments_payments.csv')

### Вспомогательные функции для препроцессинга

In [3]:
def aggregation_of_means(dataframe):
    """ Функция для агрегирования колонок, содержащих меры центральных тенденций
        (колонки, имена которых содержат постфиксы '*_MODE', '*_AVG', '*_MEDI')
    """
    df = dataframe.copy()

    cols = set(list(filter(lambda col: col.endswith(('MODE', 'AVG', 'MEDI')), df.columns)))

    # удаляем названия, содержащие категориальные признаки 
    _cat_cols = set(df[cols].select_dtypes(include=[object]).apply(pd.Series.nunique, axis=0).index.to_list())
    cols.difference_update(_cat_cols)

    cols_to_drop = copy(cols)

    results = set()
    while len(cols):
        col = cols.pop()
        prefix_name = '_'.join(col.split('_')[:-1])
        tmp = list(filter(lambda c: c.startswith(prefix_name) and c != col, cols))
        cols.difference_update(set(tmp))
        results.add((col, *tmp))
    
    results = list(map(list, results))
    
    for t in results:
        new_col_name = '_'.join(t[0].split('_')[:-1]) + '_MEAN'
        if len(t) == 1:
            df[new_col_name] = df[t]
            continue
        df[new_col_name] = df[t].mean(axis=1)
    
    df.drop(columns=cols_to_drop, inplace=True)
    return df

In [4]:
def has_documents(dataframe):
    df = dataframe.copy()
    cols = ['FLAG_DOCUMENT_{}'.format(i) for i in range(2, 22)]
    df['HAS_DOCS'] = df[cols].any(axis=1).astype('int')
    df.drop(columns=cols, inplace=True)
    return df

In [5]:
def days_birth_to_years(dataframe):
    df = dataframe.copy()
    df['AGE'] = (df['DAYS_BIRTH'].abs() / 365).astype('int')
    df.drop(columns=['DAYS_BIRTH'], inplace=True)
    return df

In [6]:
def days_employed_preprocess(dataframe):
    df = dataframe.copy()
    # определим выбросы - люди с опытом работы более 1000 лет
    outliers = df.loc[df['DAYS_EMPLOYED'] > 0, 'DAYS_EMPLOYED'].value_counts().index.to_list()  
    # заменим на значения на nan
    df['DAYS_EMPLOYED'].replace(outliers, np.nan, inplace=True)
    df['EMPLOYED_YEARS'] = df['DAYS_EMPLOYED'].abs() / 365
    df.drop(columns=['DAYS_EMPLOYED'], inplace=True)
    
    return df

In [7]:
def cnt_children_preprocess(dataframe):
    df = dataframe.copy()
    df.loc[df['CNT_CHILDREN'] > 5, 'CNT_CHILDREN'] = 6
    return df


def cnt_fam_memb_preprocess(dataframe):
    df = dataframe.copy()
    df.loc[df['CNT_FAM_MEMBERS'] > 7, 'CNT_FAM_MEMBERS'] = 8
    return df

In [8]:
def one_hot_encoder(df, categorical_columns=None, nan_as_category=False):
    """Create a new column for each categorical value in categorical columns. """
    original_columns = list(df.columns)
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    categorical_columns = [c for c in df.columns if c not in original_columns]
    return df, categorical_columns


def label_encoder(df):
    cat_features = df.select_dtypes(include=[object]).apply(pd.Series.nunique, axis=0)
    le = LabelEncoder()
    for col in cat_features[cat_features==2].index:
        df[col] = le.fit_transform(df[col].astype('str'))
    return df, df.select_dtypes(include=[object]).apply(pd.Series.nunique, axis=0).index.to_list()

## Данные по основным таблицам

In [9]:
print ('Формат тренировочной выборки', train.shape)
print ('Формат тестовой выборки', test.shape)

Формат тренировочной выборки (307511, 122)
Формат тестовой выборки (48744, 121)


In [10]:
dataset = pd.concat([train, test])
dataset.shape

(356255, 122)

In [11]:
dataset = aggregation_of_means(dataset)

dataset = has_documents(dataset)

dataset = dataset[dataset['CODE_GENDER'] != 'XNA']  # 4 человека XNA

dataset = dataset[dataset['AMT_INCOME_TOTAL'] < 20000000]  # Максимум в тестовой выборке 4 млн, в обучающей 117 млн
dataset['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)  # > 55к людей с опытом работы более 1000 лет
dataset['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)

dataset, cat_feats = label_encoder(dataset)
dataset, _ = one_hot_encoder(dataset, cat_feats)

dataset['CREDIT_TO_ANNUITY_RATIO'] = dataset['AMT_CREDIT'] / dataset['AMT_ANNUITY']
dataset['CREDIT_TO_GOODS_RATIO'] = dataset['AMT_CREDIT'] / dataset['AMT_GOODS_PRICE']

dataset['ANNUITY_TO_INCOME_RATIO'] = dataset['AMT_ANNUITY'] / dataset['AMT_INCOME_TOTAL']
dataset['CREDIT_TO_INCOME_RATIO'] = dataset['AMT_CREDIT'] / dataset['AMT_INCOME_TOTAL']
dataset['INCOME_TO_EMPLOYED_RATIO'] = dataset['AMT_INCOME_TOTAL'] / dataset['DAYS_EMPLOYED']
dataset['INCOME_TO_BIRTH_RATIO'] = dataset['AMT_INCOME_TOTAL'] / dataset['DAYS_BIRTH']

dataset['EMPLOYED_TO_BIRTH_RATIO'] = dataset['DAYS_EMPLOYED'] / dataset['DAYS_BIRTH']
dataset['ID_TO_BIRTH_RATIO'] = dataset['DAYS_ID_PUBLISH'] / dataset['DAYS_BIRTH']
dataset['CAR_TO_BIRTH_RATIO'] = dataset['OWN_CAR_AGE'] / dataset['DAYS_BIRTH']
dataset['CAR_TO_EMPLOYED_RATIO'] = dataset['OWN_CAR_AGE'] / dataset['DAYS_EMPLOYED']
dataset['PHONE_TO_BIRTH_RATIO'] = dataset['DAYS_LAST_PHONE_CHANGE'] / dataset['DAYS_BIRTH']

dataset['AMT_INCOME_TOTAL_CNT_FAM_MEMBERS_RATIO'] = dataset['AMT_INCOME_TOTAL'] / dataset['CNT_FAM_MEMBERS']
dataset['AMT_INCOME_TOTAL_CNT_CHILDREN_RATIO'] = dataset['AMT_INCOME_TOTAL'] / (1 + dataset['CNT_CHILDREN'])

# dataset['EXT_SOURCE_SUM'] = dataset[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].sum(axis=1)
# dataset['EXT_SOURCE_MUL'] = dataset['EXT_SOURCE_1'] * dataset['EXT_SOURCE_2'] * dataset['EXT_SOURCE_3']

# dataset['CNT_MISS'] = dataset.isna().sum(axis=1)

In [12]:
train = dataset.loc[dataset['TARGET'].notna(), :]
test = dataset.loc[dataset['TARGET'].isna(), :]

print ('Формат тренировочной выборки', train.shape)
print ('Формат тестовой выборки', test.shape)

Формат тренировочной выборки (307506, 206)
Формат тестовой выборки (48744, 206)


In [13]:
del dataset
gc.collect()

68

## Общие данные по кредитным бюро

In [14]:
buro.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


* CREDIT_ACTIVE - Текущий статус займа - Закрыт / Активен / Плохой / Продан (4 значения)

* CREDIT_CURRENCY - Валюта, в которой была выполнена транзакция - Currency1, Currency2, Currency3, Currency4 (4 значения)

* CREDIT_DAY_OVERDUE - Количество дней просрочки по кредиту

* CREDIT_TYPE - Consumer Credit, Credit card, Mortgage, Car loan, Microloan, Loan for working capital replemishment, Loan for Business development, Real estate loan, Unkown type of laon, Another type of loan. Cash loan, Loan for the purchase of equipment, Mobile operator loan, Interbank credit, Loan for purchase of shares (Потребительский кредит, Кредитная карта, Ипотека, Автокредит, Микрозайм, Кредит на пополнение оборотного капитала, Кредит на развитие бизнеса, Кредит на недвижимость, Неизвестный вид кредита, Другой вид кредита. Кредит наличными, Кредит на покупку оборудования, Кредит оператора мобильной связи, Межбанковский кредит, Кредит на покупку акций (15 категорий))

* DAYS_CREDIT - Количество дней, ПРОШЕДШИХ с того момента, как клиент подал заявку на кредит в отношении текущей заявки. Интерпретация - Равномерно ли распределены эти ссуды по временным интервалам? Сосредоточены ли они в одном временном интервале?

* DAYS_CREDIT_ENDDATE - Количество дней, в течение которых КРЕДИТ клиента действителен на момент подачи заявки

* CREDIT_DAY_OVERDUE - Количество дней, в течение которых КРЕДИТ клиента превышает дату окончания на момент подачи заявки

* AMT_CREDIT_SUM - Общий доступный кредит для клиента 

* AMT_CREDIT_SUM_DEBT - Общая сумма еще не погашена 

* AMT_CREDIT_SUM_LIMIT - Текущий кредит, который был использован 

* AMT_CREDIT_SUM_OVERDUE - Текущий кредитный платеж, который просрочен 

* CNT_CREDIT_PROLONG - Сколько раз продлевалась дата кредитования

### 1) Количество прошлых кредитов по клиентам

In [15]:
grp = buro[['SK_ID_CURR', 'DAYS_CREDIT']].groupby(by=['SK_ID_CURR'])['DAYS_CREDIT'].count().reset_index().rename(index=str, columns={'DAYS_CREDIT': 'BUREAU_LOAN_COUNT'})
buro = buro.merge(grp, on=['SK_ID_CURR'], how='left')
buro.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_LOAN_COUNT
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,11
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,11
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,11
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,11


### 2) Количество видов прошлых кредитов

In [16]:
grp = buro[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by=['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(index=str, columns={'CREDIT_TYPE': 'BUREAU_LOAN_TYPES'})
buro = buro.merge(grp, on=['SK_ID_CURR'], how='left')
buro.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_LOAN_COUNT,BUREAU_LOAN_TYPES
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,3
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,11,3
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,11,3
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,11,3
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,11,3


### 3) Среднее количество кредитов по каждой категории

Ориентирован ли клиент в получении нескольких типов кредитов или ориентирован на один тип


In [17]:
buro['AVERAGE_LOAN_TYPE'] = buro['BUREAU_LOAN_COUNT'] / buro['BUREAU_LOAN_TYPES']
buro.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_LOAN_COUNT,BUREAU_LOAN_TYPES,AVERAGE_LOAN_TYPE
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,3,3.666667
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,11,3,3.666667
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,11,3,3.666667
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,11,3,3.666667
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,11,3,3.666667


### 4) % Активных кредитов по данным из бюро

In [18]:
buro['CREDIT_ACTIVE_BINARY'] = buro['CREDIT_ACTIVE']

buro['CREDIT_ACTIVE_BINARY'] = buro['CREDIT_ACTIVE_BINARY'].apply(lambda x: 0 if x == 'Closed' else 1)

# Calculate mean number of loans that are ACTIVE per CUSTOMER 
grp = buro.groupby(by=['SK_ID_CURR'])['CREDIT_ACTIVE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ACTIVE_BINARY': 'ACTIVE_LOANS_PERCENTAGE'})
buro = buro.merge(grp, on=['SK_ID_CURR'], how='left')
del buro['CREDIT_ACTIVE_BINARY']
buro.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_LOAN_COUNT,BUREAU_LOAN_TYPES,AVERAGE_LOAN_TYPE,ACTIVE_LOANS_PERCENTAGE
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,3,3.666667,0.545455
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,11,3,3.666667,0.545455
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,11,3,3.666667,0.545455
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,11,3,3.666667,0.545455
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,11,3,3.666667,0.545455


### 5) СРЕДНЕЕ ЧИСЛО ДНЕЙ МЕЖДУ УСПЕШНЫМИ ЗАЯВКАМИ ДЛЯ КАЖДОГО КЛИЕНТА

Как часто клиент брал кредит в прошлом? Распределялись ли они через равные промежутки времени - сигнал о хорошем финансовом планировании ИЛИ были кредиты, сконцентрированы на меньшем временном интервале, что указывало на потенциальные финансовые проблемы?

In [19]:
# Groupby each Customer and Sort values of DAYS_CREDIT in ascending order
grp = buro[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT']].groupby(by=['SK_ID_CURR'])
grp1 = grp.apply(lambda x: x.sort_values(['DAYS_CREDIT'], ascending=False)).reset_index(drop = True)

# Calculate Difference between the number of Days 
grp1['DAYS_CREDIT1'] = grp1['DAYS_CREDIT'] * -1
grp1['DAYS_DIFF'] = grp1.groupby(by=['SK_ID_CURR'])['DAYS_CREDIT1'].diff()
grp1['DAYS_DIFF'] = grp1['DAYS_DIFF'].fillna(0).astype('uint32')
del grp1['DAYS_CREDIT1'], grp1['DAYS_CREDIT'], grp1['SK_ID_CURR'], grp
gc.collect()

buro = buro.merge(grp1, on=['SK_ID_BUREAU'], how='left')
buro.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_LOAN_COUNT,BUREAU_LOAN_TYPES,AVERAGE_LOAN_TYPE,ACTIVE_LOANS_PERCENTAGE,DAYS_DIFF
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,3,3.666667,0.545455,224
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,11,3,3.666667,0.545455,5
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,11,3,3.666667,0.545455,160
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,11,3,3.666667,0.545455,0
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,11,3,3.666667,0.545455,132


### 6) % КРЕДИТОВ НА КЛИЕНТА, ГДЕ КОНЕЧНАЯ ДАТА ДЛЯ КРЕДИТА ПРОШЛА

ОТРИЦАТЕЛЬНОЕ ЗНАЧЕНИЕ - на момент подачи заявки дата кредита была в прошлом (потенциальный красный флаг !!!)

ПОЛОЖИТЕЛЬНАЯ СТОИМОСТЬ - Дата зачисления находится в будущем на момент подачи заявки (потенциальный хороший знак !!!!)

In [20]:
buro['CREDIT_ENDDATE_BINARY'] = buro['DAYS_CREDIT_ENDDATE']

buro['CREDIT_ENDDATE_BINARY'] = buro['CREDIT_ENDDATE_BINARY'].apply(lambda x: 0 if x < 0 else 1)

grp = buro.groupby(by=['SK_ID_CURR'])['CREDIT_ENDDATE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ENDDATE_BINARY': 'CREDIT_ENDDATE_PERCENTAGE'})
buro = buro.merge(grp, on=['SK_ID_CURR'], how='left')
del buro['CREDIT_ENDDATE_BINARY']
buro.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_LOAN_COUNT,BUREAU_LOAN_TYPES,AVERAGE_LOAN_TYPE,ACTIVE_LOANS_PERCENTAGE,DAYS_DIFF,CREDIT_ENDDATE_PERCENTAGE
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,3,3.666667,0.545455,224,0.545455
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,11,3,3.666667,0.545455,5,0.545455
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,11,3,3.666667,0.545455,160,0.545455
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,11,3,3.666667,0.545455,0,0.545455
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,11,3,3.666667,0.545455,132,0.545455


### 7) СРЕДНЕЕ КОЛИЧЕСТВО ДНЕЙ, В КОТОРЫЕ истекает срок действия кредита в будущем

In [21]:
# # Dummy column to calculate 1 or 0 values. 1 for Positive CREDIT_ENDDATE and 0 for Negative
# buro['CREDIT_ENDDATE_BINARY'] = buro['DAYS_CREDIT_ENDDATE']


# buro['CREDIT_ENDDATE_BINARY'] = buro['CREDIT_ENDDATE_BINARY'].apply(lambda x: 0 if x < 0 else 1)

# # We take only positive values of  ENDDATE since we are looking at Bureau Credit VALID IN FUTURE 
# # as of the date of the customer's loan application with Home Credit 
# buro1 = buro[buro['CREDIT_ENDDATE_BINARY'] == 1]

# #Calculate Difference in successive future end dates of CREDIT 

# # Create Dummy Column for CREDIT_ENDDATE 
# buro1['DAYS_CREDIT_ENDDATE1'] = buro1['DAYS_CREDIT_ENDDATE']
# # Groupby Each Customer ID 
# grp = buro1[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT_ENDDATE1']].groupby(by=['SK_ID_CURR'])
# # Sort the values of CREDIT_ENDDATE for each customer ID 
# grp1 = grp.apply(lambda x: x.sort_values(['DAYS_CREDIT_ENDDATE1'], ascending=True)).reset_index(drop=True)
# del grp
# gc.collect()

# # Calculate the Difference in ENDDATES and fill missing values with zero 
# grp1['DAYS_ENDDATE_DIFF'] = grp1.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT_ENDDATE1'].diff()
# grp1['DAYS_ENDDATE_DIFF'] = grp1['DAYS_ENDDATE_DIFF'].fillna(0).astype('uint32')
# del grp1['DAYS_CREDIT_ENDDATE1'], grp1['SK_ID_CURR']
# gc.collect()
# print("Difference days calculated")

# # Merge new feature 'DAYS_ENDDATE_DIFF' with original Data frame for BUREAU DATA
# buro = buro.merge(grp1, on = ['SK_ID_BUREAU'], how='left')
# del grp1
# gc.collect()

# # Calculate Average of DAYS_ENDDATE_DIFF

# grp = buro[['SK_ID_CURR', 'DAYS_ENDDATE_DIFF']].groupby(by=['SK_ID_CURR'])['DAYS_ENDDATE_DIFF'].mean().reset_index().rename(index=str, columns={'DAYS_ENDDATE_DIFF': 'AVG_ENDDATE_FUTURE'})
# buro = buro.merge(grp, on=['SK_ID_CURR'], how='left')
# del grp 
# #del B['DAYS_ENDDATE_DIFF']
# del buro['CREDIT_ENDDATE_BINARY'], buro['DAYS_CREDIT_ENDDATE']
# gc.collect()
# buro.head()

### 8) ОТНОШЕНИЕ ЗАДОЛЖЕННОСТЬ К КРЕДИТУ

Отношение общей задолженности к общей сумме кредита для каждого клиента

Высокое значение может указывать на проблемные долги.

In [22]:
# buro['AMT_CREDIT_SUM_DEBT'] = buro['AMT_CREDIT_SUM_DEBT'].fillna(0)
# buro['AMT_CREDIT_SUM'] = buro['AMT_CREDIT_SUM'].fillna(0)

# grp1 = buro[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename(index=str, columns={ 'AMT_CREDIT_SUM_DEBT': 'TOTAL_CUSTOMER_DEBT'})
# grp2 = buro[['SK_ID_CURR', 'AMT_CREDIT_SUM']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM'].sum().reset_index().rename(index=str, columns={ 'AMT_CREDIT_SUM': 'TOTAL_CUSTOMER_CREDIT'})

# buro = buro.merge(grp1, on=['SK_ID_CURR'], how='left')
# buro = buro.merge(grp2, on=['SK_ID_CURR'], how='left')
# del grp1, grp2
# gc.collect()

# buro['DEBT_CREDIT_RATIO'] = buro['TOTAL_CUSTOMER_DEBT'] / buro['TOTAL_CUSTOMER_CREDIT']

# del buro['TOTAL_CUSTOMER_DEBT'], buro['TOTAL_CUSTOMER_CREDIT']
# gc.collect()
# buro.head()

### 9) OVERDUE OVER DEBT RATIO
What fraction of total Debt is overdue per customer?
A high value could indicate a potential DEFAULT

In [23]:
# buro['AMT_CREDIT_SUM_DEBT'] = buro['AMT_CREDIT_SUM_DEBT'].fillna(0)
# buro['AMT_CREDIT_SUM_OVERDUE'] = buro['AMT_CREDIT_SUM_OVERDUE'].fillna(0)

# grp1 = buro[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename(index=str, columns={ 'AMT_CREDIT_SUM_DEBT': 'TOTAL_CUSTOMER_DEBT'})
# grp2 = buro[['SK_ID_CURR', 'AMT_CREDIT_SUM_OVERDUE']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_OVERDUE'].sum().reset_index().rename(index=str, columns={ 'AMT_CREDIT_SUM_OVERDUE': 'TOTAL_CUSTOMER_OVERDUE'})

# buro = buro.merge(grp1, on=['SK_ID_CURR'], how='left')
# buro = buro.merge(grp2, on=['SK_ID_CURR'], how='left')
# del grp1, grp2
# gc.collect()

# buro['OVERDUE_DEBT_RATIO'] = buro['TOTAL_CUSTOMER_OVERDUE'] / buro['TOTAL_CUSTOMER_DEBT']

# del buro['TOTAL_CUSTOMER_OVERDUE'], buro['TOTAL_CUSTOMER_DEBT']
# gc.collect()
# buro.head()

### 10) AVERAGE NUMBER OF LOANS PROLONGED

In [24]:
buro['CNT_CREDIT_PROLONG'] = buro['CNT_CREDIT_PROLONG'].fillna(0)
grp = buro[['SK_ID_CURR', 'CNT_CREDIT_PROLONG']].groupby(by=['SK_ID_CURR'])['CNT_CREDIT_PROLONG'].mean().reset_index().rename(index=str, columns={'CNT_CREDIT_PROLONG': 'AVG_CREDITDAYS_PROLONGED'})
buro = buro.merge(grp, on=['SK_ID_CURR'], how='left')
buro.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_LOAN_COUNT,BUREAU_LOAN_TYPES,AVERAGE_LOAN_TYPE,ACTIVE_LOANS_PERCENTAGE,DAYS_DIFF,CREDIT_ENDDATE_PERCENTAGE,AVG_CREDITDAYS_PROLONGED
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,3,3.666667,0.545455,224,0.545455,0.0
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,11,3,3.666667,0.545455,5,0.545455,0.0
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,11,3,3.666667,0.545455,160,0.545455,0.0
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,11,3,3.666667,0.545455,0,0.545455,0.0
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,11,3,3.666667,0.545455,132,0.545455,0.0


In [25]:
buro_cat_features = buro.select_dtypes(include=[object]).apply(pd.Series.nunique, axis=0)
buro, _ = one_hot_encoder(buro, categorical_columns=buro_cat_features.index.to_list())
buro.head()

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_LOAN_COUNT,BUREAU_LOAN_TYPES,AVERAGE_LOAN_TYPE,ACTIVE_LOANS_PERCENTAGE,DAYS_DIFF,CREDIT_ENDDATE_PERCENTAGE,AVG_CREDITDAYS_PROLONGED,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,CREDIT_TYPE_Cash loan (non-earmarked),CREDIT_TYPE_Consumer credit,CREDIT_TYPE_Credit card,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan
0,215354,5714462,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,-131,NaN,11,3,3.666667,0.545455,224,0.545455,0.0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,-20,NaN,11,3,3.666667,0.545455,5,0.545455,0.0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,-16,NaN,11,3,3.666667,0.545455,160,0.545455,0.0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,-16,NaN,11,3,3.666667,0.545455,0,0.545455,0.0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,215354,5714466,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,-21,NaN,11,3,3.666667,0.545455,132,0.545455,0.0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [26]:

avg_buro = buro.groupby('SK_ID_CURR').mean()
avg_buro['BUREAU_CNT'] = buro[['SK_ID_BUREAU', 'SK_ID_CURR']].groupby('SK_ID_CURR').count()['SK_ID_BUREAU']

del buro
gc.collect()

11

## Данные кредитного бюро по ежемесячному балансу кредитов

In [27]:
buro_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


MONTHS_BALANCE - количество месяцев до даты подачи заявки на кредит

In [28]:
buro_balance.STATUS.value_counts()

C    13646993
0     7499507
X     5810482
1      242347
5       62406
2       23419
3        8924
4        5847
Name: STATUS, dtype: int64

С - closed, то есть погашенный кредит. X - неизвестный статус. 0 - текущий кредит, отсуствие просрочек. 1 - просрочка 1-30 дней, 2 - просрочка 31-60 дней и так далее до статуса 5 - кредит продан третьей стороне или списан

In [29]:
buro_grouped_size = buro_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].size()
buro_grouped_max = buro_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].max()
buro_grouped_min = buro_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].min()

buro_counts = buro_balance.groupby('SK_ID_BUREAU')['STATUS'].value_counts(normalize = False)
buro_counts_unstacked = buro_counts.unstack('STATUS')
buro_counts_unstacked.columns = ['STATUS_0', 'STATUS_1','STATUS_2','STATUS_3','STATUS_4','STATUS_5','STATUS_C','STATUS_X',]
buro_counts_unstacked['MONTHS_COUNT'] = buro_grouped_size
buro_counts_unstacked['MONTHS_MIN'] = buro_grouped_min
buro_counts_unstacked['MONTHS_MAX'] = buro_grouped_max

avg_buro = avg_buro.join(buro_counts_unstacked, how='left', on='SK_ID_BUREAU')
del avg_buro['SK_ID_BUREAU']
del buro_balance
gc.collect()
avg_buro.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:1097: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  UserWarning,


,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_LOAN_COUNT,BUREAU_LOAN_TYPES,AVERAGE_LOAN_TYPE,ACTIVE_LOANS_PERCENTAGE,DAYS_DIFF,CREDIT_ENDDATE_PERCENTAGE,AVG_CREDITDAYS_PROLONGED,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,CREDIT_TYPE_Cash loan (non-earmarked),CREDIT_TYPE_Consumer credit,CREDIT_TYPE_Credit card,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,BUREAU_CNT,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X,MONTHS_COUNT,MONTHS_MIN,MONTHS_MAX
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,-735.000000,0.0,82.428571,-825.500000,NaN,0.0,207623.571429,85240.928571,0.00000,0.0,-93.142857,3545.357143,7.0,1.0,7.0,0.428571,217.571429,0.428571,0.0,0.428571,0.0,0.571429,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,1.0,NaN,NaN,NaN,NaN,NaN,44.0,7.0,52.0,-51.0,0.0
100002,-874.000000,0.0,-349.000000,-697.500000,1681.029,0.0,108131.945625,49156.200000,7997.14125,0.0,-499.875000,0.000000,8.0,2.0,4.0,0.250000,166.750000,0.625000,0.0,0.250000,0.0,0.750000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,-1400.750000,0.0,-544.500000,-1097.333333,0.000,0.0,254350.125000,0.000000,202500.00000,0.0,-816.000000,NaN,4.0,2.0,2.0,0.250000,495.000000,0.250000,0.0,0.250000,0.0,0.750000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004,-867.000000,0.0,-488.500000,-532.500000,0.000,0.0,94518.900000,0.000000,0.00000,0.0,-532.000000,NaN,2.0,1.0,2.0,0.000000,459.000000,0.000000,0.0,0.000000,0.0,1.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100005,-190.666667,0.0,439.333333,-123.000000,0.000,0.0,219042.000000,189469.500000,0.00000,0.0,-54.333333,1420.500000,3.0,2.0,1.5,0.666667,103.666667,0.666667,0.0,0.666667,0.0,0.333333,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,-4.0,0.0


## Данные по предыдущим заявкам

In [30]:
prev.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
prev_cat_features = prev.select_dtypes(include=[object]).apply(pd.Series.nunique, axis=0)
prev_cat_features

NAME_CONTRACT_TYPE              4
WEEKDAY_APPR_PROCESS_START      7
FLAG_LAST_APPL_PER_CONTRACT     2
NAME_CASH_LOAN_PURPOSE         25
NAME_CONTRACT_STATUS            4
NAME_PAYMENT_TYPE               4
CODE_REJECT_REASON              9
NAME_TYPE_SUITE                 7
NAME_CLIENT_TYPE                4
NAME_GOODS_CATEGORY            28
NAME_PORTFOLIO                  5
NAME_PRODUCT_TYPE               3
CHANNEL_TYPE                    8
NAME_SELLER_INDUSTRY           11
NAME_YIELD_GROUP                5
PRODUCT_COMBINATION            17
dtype: int64

In [32]:
le = LabelEncoder()
for col in prev_cat_features[prev_cat_features < 4].index:
    prev[col] = le.fit_transform(prev[col].astype('str'))

prev = pd.get_dummies(prev, columns=prev_cat_features[prev_cat_features >=4].index.to_list())
avg_prev = prev.groupby('SK_ID_CURR').mean()
cnt_prev = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
avg_prev['nb_app'] = cnt_prev['SK_ID_PREV']
del avg_prev['SK_ID_PREV']

del prev
gc.collect()

0

## Баланс по кредитной карте

In [33]:
POS_CASH.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [34]:
POS_CASH.NAME_CONTRACT_STATUS.value_counts()

Active                   9151119
Completed                 744883
Signed                     87260
Demand                      7065
Returned to the store       5461
Approved                    4917
Amortized debt               636
Canceled                      15
XNA                            2
Name: NAME_CONTRACT_STATUS, dtype: int64

In [35]:
le = LabelEncoder()
POS_CASH['NAME_CONTRACT_STATUS'] = le.fit_transform(POS_CASH['NAME_CONTRACT_STATUS'].astype(str))
nunique_status = POS_CASH[['SK_ID_CURR', 'NAME_CONTRACT_STATUS']].groupby('SK_ID_CURR').nunique()
nunique_status2 = POS_CASH[['SK_ID_CURR', 'NAME_CONTRACT_STATUS']].groupby('SK_ID_CURR').max()
POS_CASH['NUNIQUE_STATUS'] = nunique_status['NAME_CONTRACT_STATUS']
POS_CASH['NUNIQUE_STATUS2'] = nunique_status2['NAME_CONTRACT_STATUS']
POS_CASH.drop(['SK_ID_PREV', 'NAME_CONTRACT_STATUS'], axis=1, inplace=True)

## Данные по картам

In [36]:
credit_card.head(7)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
5,2646502,380010,-7,82903.815,270000,0.0,0.0,0.0,0.0,4449.105,3825.0,3825.0,80519.040,82773.315,82773.315,0.0,0,0.0,0.0,2.0,Active,7,0
6,1079071,171320,-6,353451.645,585000,67500.0,67500.0,0.0,0.0,14684.175,15750.0,15750.0,345433.860,351881.145,351881.145,1.0,1,0.0,0.0,6.0,Active,0,0


In [37]:
credit_card['NAME_CONTRACT_STATUS'] = le.fit_transform(credit_card['NAME_CONTRACT_STATUS'].astype(str))
nunique_status = credit_card[['SK_ID_CURR', 'NAME_CONTRACT_STATUS']].groupby('SK_ID_CURR').nunique()
nunique_status2 = credit_card[['SK_ID_CURR', 'NAME_CONTRACT_STATUS']].groupby('SK_ID_CURR').max()
credit_card['NUNIQUE_STATUS'] = nunique_status['NAME_CONTRACT_STATUS']
credit_card['NUNIQUE_STATUS2'] = nunique_status2['NAME_CONTRACT_STATUS']
credit_card.drop(['SK_ID_PREV', 'NAME_CONTRACT_STATUS'], axis=1, inplace=True)

## Данные по платежам

In [38]:
payments.head(7)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585
5,1137312,164489,1.0,12,-1384.0,-1417.0,5970.375,5970.375
6,2234264,184693,4.0,11,-349.0,-352.0,29432.295,29432.295


In [39]:
avg_payments = payments.groupby('SK_ID_CURR').mean()
avg_payments2 = payments.groupby('SK_ID_CURR').max()
avg_payments3 = payments.groupby('SK_ID_CURR').min()
del avg_payments['SK_ID_PREV']

del payments
gc.collect()

22

## Объединение таблиц

In [40]:
train = train.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')

train = train.merge(right=avg_buro.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_buro.reset_index(), how='left', on='SK_ID_CURR')

train = train.merge(POS_CASH.groupby('SK_ID_CURR').mean().reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(POS_CASH.groupby('SK_ID_CURR').mean().reset_index(), how='left', on='SK_ID_CURR')

train = train.merge(credit_card.groupby('SK_ID_CURR').mean().reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(credit_card.groupby('SK_ID_CURR').mean().reset_index(), how='left', on='SK_ID_CURR')

train = train.merge(right=avg_payments.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_payments.reset_index(), how='left', on='SK_ID_CURR')

train = train.merge(right=avg_payments2.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_payments2.reset_index(), how='left', on='SK_ID_CURR')

train = train.merge(right=avg_payments3.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_payments3.reset_index(), how='left', on='SK_ID_CURR')

del avg_prev, avg_buro, POS_CASH, credit_card, avg_payments, avg_payments2, avg_payments3
gc.collect()

0

In [41]:
Y = train['TARGET']
test_id = test['SK_ID_CURR']

test.drop(columns=['TARGET'], inplace=True)
train.drop(columns=['TARGET'], inplace=True)
del train['SK_ID_CURR'], test['SK_ID_CURR']
print ('Формат тренировочной выборки', train.shape)
print ('Формат тестовой выборки', test.shape)
print ('Формат целевого столбца', Y.shape)

Формат тренировочной выборки (307506, 467)
Формат тестовой выборки (48744, 467)
Формат целевого столбца (307506,)


In [42]:
feats = list(train.columns)

In [43]:
train.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan, inplace=True)

In [44]:
# пропущенные значения
imputer = SimpleImputer(strategy='median')
train = imputer.fit_transform(train)
test = imputer.transform(test)

# стандартизация
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

train = pd.DataFrame(train, columns=feats)
test = pd.DataFrame(test, columns=feats)

## Обучение моодели


В качестве классификатора возьмём градиентный бустинг

In [45]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
folds = KFold(n_splits=5, shuffle=True, random_state=23)
val_preds = np.zeros(train.shape[0])
predictions = np.zeros(test.shape[0])

feature_importance_df = pd.DataFrame({'Features': feats, 'Importance': np.zeros(len(feats))})
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train)):
    trn_x, trn_y = train[feats].iloc[trn_idx], Y.iloc[trn_idx]
    val_x, val_y = train[feats].iloc[val_idx], Y.iloc[val_idx]
    
    clf = LGBMClassifier(
        random_state=23,
        n_estimators=1000,
        learning_rate=0.05,
        num_leaves=34,
        max_depth=8,
        verbose=-1,
    )
    
    clf.fit(trn_x, trn_y, 
            eval_set= [(trn_x, trn_y), (val_x, val_y)], 
            eval_metric='roc_auc', verbose=100, early_stopping_rounds=100)
    
    val_preds[val_idx] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]
    predictions += clf.predict_proba(test[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
    
    feature_importance_df["Importance"] += clf.feature_importances_ / folds.n_splits
    
    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(val_y, val_preds[val_idx])))
    
    del clf, trn_x, trn_y, val_x, val_y
    gc.collect()

print('Full AUC score %.6f' % roc_auc_score(Y, val_preds))

Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.232635	valid_1's binary_logloss: 0.24309
[200]	training's binary_logloss: 0.221279	valid_1's binary_logloss: 0.239951
[300]	training's binary_logloss: 0.212935	valid_1's binary_logloss: 0.239413
[400]	training's binary_logloss: 0.205568	valid_1's binary_logloss: 0.239169
[500]	training's binary_logloss: 0.198939	valid_1's binary_logloss: 0.238965
[600]	training's binary_logloss: 0.192805	valid_1's binary_logloss: 0.238907
[700]	training's binary_logloss: 0.187019	valid_1's binary_logloss: 0.238814
Early stopping, best iteration is:
[691]	training's binary_logloss: 0.187489	valid_1's binary_logloss: 0.238792
Fold  1 AUC : 0.784771
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.232839	valid_1's binary_logloss: 0.241931
[200]	training's binary_logloss: 0.221433	valid_1's binary_logloss: 0.239114
[300]	training's binary_logloss: 0.212945	valid

In [46]:
# Датафрейм для загрузки
result_subm = pd.DataFrame()
result_subm['SK_ID_CURR'] = test_id
result_subm['TARGET'] = predictions

# Сохранение датафрейма
result_subm.to_csv('{}_full_data_FE.csv'.format('LGBM'), index = False)

Результат на тестовой выборке **0.78952**

## Удаление неинформативных признаков

In [47]:
non_importance_features = feature_importance_df[feature_importance_df['Importance'] == 0]['Features'].to_list()


In [48]:
cols_to_drop = list(filter(lambda c: c.startswith('SK_ID'), train.columns))
cols_to_drop += non_importance_features

In [49]:
new_train = train.drop(columns=cols_to_drop)
new_test = test.drop(columns=cols_to_drop)

new_feats = list(new_train.columns)
# print(new_feats)

In [50]:
folds = KFold(n_splits=5, shuffle=True, random_state=23)
val_preds = np.zeros(new_train.shape[0])
predictions = np.zeros(new_test.shape[0])

feature_importance_df = pd.DataFrame({'Features': new_feats, 'Importance': np.zeros(len(new_feats))})
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(new_train)):
    trn_x, trn_y = new_train[new_feats].iloc[trn_idx], Y.iloc[trn_idx]
    val_x, val_y = new_train[new_feats].iloc[val_idx], Y.iloc[val_idx]
    
    clf = LGBMClassifier(
        random_state=23,
        n_estimators=1000,
        learning_rate=0.05,
        num_leaves=34,
        max_depth=8,
        verbose=-1,
    )
    
    clf.fit(trn_x, trn_y, 
            eval_set= [(trn_x, trn_y), (val_x, val_y)], 
            eval_metric='roc_auc', verbose=100, early_stopping_rounds=100)
    
    val_preds[val_idx] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]
    predictions += clf.predict_proba(test[new_feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
    
    feature_importance_df["Importance"] += clf.feature_importances_ / folds.n_splits
    
    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(val_y, val_preds[val_idx])))
    
    del clf, trn_x, trn_y, val_x, val_y
    gc.collect()

print('Full AUC score %.6f' % roc_auc_score(Y, val_preds))

result_subm = pd.DataFrame()
result_subm['SK_ID_CURR'] = test_id
result_subm['TARGET'] = predictions
result_subm.to_csv('{}_full_data_FE_most_importance_feats.csv'.format('LGBM'), index = False)

Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.232654	valid_1's binary_logloss: 0.243114
[200]	training's binary_logloss: 0.221351	valid_1's binary_logloss: 0.239902
[300]	training's binary_logloss: 0.213061	valid_1's binary_logloss: 0.239268
[400]	training's binary_logloss: 0.205685	valid_1's binary_logloss: 0.238989
[500]	training's binary_logloss: 0.199016	valid_1's binary_logloss: 0.23868
[600]	training's binary_logloss: 0.192901	valid_1's binary_logloss: 0.238634
[700]	training's binary_logloss: 0.187041	valid_1's binary_logloss: 0.23851
Early stopping, best iteration is:
[683]	training's binary_logloss: 0.18795	valid_1's binary_logloss: 0.238457
Fold  1 AUC : 0.786213
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.232767	valid_1's binary_logloss: 0.24186
[200]	training's binary_logloss: 0.221327	valid_1's binary_logloss: 0.239063
[300]	training's binary_logloss: 0.212942	valid_1'

Результат на тестовой выборке **0.78943**